Stage 1: Authenticate GovWin Connection and obtain a access token.

In [1]:
import requests

client_id = "H859MH8H3DN5A03S7N6MU702E3SMCFF6F30E2L40QG58G"
client_secret = "9NH77FAHV5LKP2MBADA6LL8GLJEPUPTNTVVRLTO74GMQ7"
url = f"https://services.govwin.com/neo-ws/oauth/token?client_id={client_id}&client_secret={client_secret}&grant_type=password&username=webservices4@bah.com&password=Testing05302023&scope=read"

data = {"key1": "value1", "key2": "value2"}

response = requests.post(url, data=data)

# Check the response status code
if response.status_code == 200:
    print("POST request was successful, connection established.")

    access_token = response.json().get("access_token")
    print("Bearer token:", access_token)
    
else:
    print("POST request failed. Status code:", response.status_code)

# Print the response content
# print(response.text)

POST request was successful, connection established.
Bearer token: 63a07810-f91a-4923-a274-7ac530e95931


Stage 2: Create enpoint requests and streamline saved search results into json/txt files.

In [15]:
import requests

# url = "https://services.govwin.com/neo-ws/opportunities?savedSearchId=1237859"
# url = "https://services.govwin.com/neo-ws/opportunities"

# Set the bearer token
bearer_token = access_token

# Set your desired search parameters
# search_params = {
#     'oppSelectionDateFrom': '01/01/2020',
#     'max': 100,
#     'offset': 100
# }

# Define API endpoint and saved search ID
# Search by saved searches, create a list of opp_ids
# endpoint = "https://services.govwin.com/neo-ws/opportunities?governmentEntityIds=131088"
# endpoint = "https://services.govwin.com/neo-ws/opportunities?naics=541511"
endpoint = "https://services.govwin.com/neo-ws/opportunities?savedSearchId=1236146"

# Search by opportunity ID, 3 letter prefix + ID number, i.e .../neo-ws/opportunities/OPP178753,OPP135165
opp_id = ['178753', '35451', '135165', '229319', '85461', '228608', '142729', '61561', '202410', '135006']
opp_id = [f"OPP{i}" for i in opp_id] 
opp_id = ",".join(opp_id) 
endpoint = f"https://services.govwin.com/neo-ws/opportunities/{opp_id}"

# Make the API request
headers = {"Authorization": f"Bearer {bearer_token}"}
response = requests.get(endpoint, headers=headers)
# response1 = requests.get(endpoint, headers=headers, params={"oppSelectionDateFrom": '01/01/1900', "max": 100, "offset": 0})
# response2 = requests.get(endpoint, headers=headers, params={"oppSelectionDateFrom": '01/01/1900', "max": 100, "offset": 100})

# Check the response status code
if response.status_code == 200:
    print("GET request was successful, endpoint collection saved to artifacts folder.")
else:
    print("GET request failed. Status code:", response.status_code)

# Print the response content
print(response.text)

#save response.text to a text file
with open(f'artifacts/govwin_extract4.json', 'w') as file:
    file.write(response.text)

#save response.text to a text file
with open(f'artifacts/govwin_extract4.txt', 'w') as file:
    file.write(response.text)

GET request was successful, endpoint collection saved to artifacts folder.
{"links":{},"opportunities":[{"links":{"webHref":{"href":"https://iq.govwin.com/neo/opportunity/view/35451"},"contacts":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/contacts"},"relatedDocuments":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/relatedDocuments"},"markedByUsers":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/markedByUsers"},"placesOfPerformance":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/placesOfPerformance"},"govEntity":{"href":"https://services.govwin.com/neo-ws/govEntities/129650"},"fboNotices":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/fboNotices"},"relatedArticles":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/relatedArticles"},"contractVehicles":{"href":"https://services.govwin.com/neo-ws/opportunities/OPP35451/contractVehicles"},"verticals":{"href":"https://servi

Stage 3: JSON search and feature extraction.

In [8]:
import json

with open('search_items.txt', 'r') as file:
    search_items = [line.strip() for line in file.readlines()]

# load json file
with open('artifacts/govwin_extract.json', 'r') as file:
    data = json.load(file)

# scrape desired information based on search items txt file
scraped_data = {}
for item in search_items:
    if item in data:
        scraped_data[item] = data['opportunities'][0][item]

#print scraped data
for item, value in scraped_data.items():
    print(f"{item}: {value}")

#save scraped data to a json file
with open(f'artifacts/govwin_extract_scraped.txt', 'w') as file:
    json.dump(scraped_data, file, indent=4)

In [10]:
import json

# Read the search items from the text file
with open('search_items.txt', 'r') as file:
    search_items = [line.strip() for line in file.readlines()]

# Load the JSON data
with open('artifacts\govwin_extract.json', 'r') as file:
    data = json.load(file)

# Scrape the desired information based on the search items
scraped_data = {}
opportunity = data['opportunities'][0]

for item in search_items:
    if item in opportunity:
        scraped_data[item] = opportunity[item]
    elif item in opportunity['govEntity']:
        scraped_data[item] = opportunity['govEntity'][item]
    elif item in opportunity['primaryNAICS']:
        scraped_data[item] = opportunity['primaryNAICS'][item]

# Print the scraped data
for item, value in scraped_data.items():
    print(f"{item}: {value}")

